## Factor calculation
We have airbnb apartment prices for 80 cities. From numbeo we have for about 500 cities prices for 1 one-bedroom apartment in city center.
We compare airbnb prices and numbeo prices for month/30 = day/rent. We divide the airbnb price and day/rent price. Then we apply the descriptive analytic function, exclusion the outliers and then we can determine the factor. Than we can keep the values from airbnb list and convert rest values from numbeo via factor on the level of airbnb prices.

In [9]:
# import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sql_functions import get_dataframe #

In [11]:
#Get the airbnb data from sql
schema = 'capstone_travel_index'
airbnb_df = get_dataframe(f'SELECT * FROM {schema}.airbnb_prices_all')

In [14]:
#Check the airbnb data
airbnb_df.head()

,Entire home/apt,Hotel room,Private room,Shared room,city
0,248.898913,177.322581,150.766495,108.230769,Amsterdam
1,126.555447,148.000000,90.354839,65.000000,Antwerp
2,164.000244,404.632800,124.963620,59.880229,Asheville
3,95.573357,331.087591,102.103448,39.728395,Athens
4,310.408633,720.033600,161.581835,65.029578,Austin


In [15]:
#Get the numbeo apartment costs data from sql
schema = 'capstone_travel_index'
numbeo_df = get_dataframe(f'SELECT * FROM {schema}.city_apart_prices')

In [16]:
#Check the numbeo data
numbeo_df.head()

,average_price,item_name,city,country,price_per_day
0,563.909091,"Apartment (1 bedroom) in City Centre, Rent Per...",Aachen,Germany,18.796970
1,726.513820,"Apartment (1 bedroom) in City Centre, Rent Per...",Aalborg,Denmark,24.217127
2,644.923561,"Apartment (1 bedroom) in City Centre, Rent Per...",Aberdeen,United Kingdom,21.497452
3,491.224639,"Apartment (1 bedroom) in City Centre, Rent Per...",Abidjan,Ivory Coast,16.374155
4,641.334624,"Apartment (1 bedroom) in City Centre, Rent Per...",Accra,Ghana,21.377821


In [17]:
#Drop the columns in airbnb dataset: 'Hotel room', 'Private room', 'Shared room'. Now we have columns with prices for entire home/apartment and city.
airbnb_df = airbnb_df.drop(['Hotel room', 'Private room', 'Shared room'], axis = 1)

In [18]:
#Info about airbnb data: 86 non-null values
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Entire home/apt  86 non-null     float64
 1   city             86 non-null     object 
dtypes: float64(1), object(1)
memory usage: 1.5+ KB


In [19]:
#Drop the columns in numbeo dataset: 'average_price', 'item_name'. Now we have columns with cities, countries and price_per_day.
numbeo_df = numbeo_df.drop(['average_price', 'item_name'], axis = 1)

In [21]:
numbeo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           525 non-null    object 
 1   country        525 non-null    object 
 2   price_per_day  525 non-null    float64
dtypes: float64(1), object(2)
memory usage: 12.4+ KB


In [22]:
numbeo_df['city'].nunique()

520

In [23]:
#Merge airbnb and numbeo data on city to became a data for the same cities and storage the data in merged_df

merged_df = pd.merge(airbnb_df, numbeo_df, on='city')

In [30]:
#Check the data
merged_df.head(10)

,Entire home/apt,city,country,price_per_day,factor
0,248.898913,Amsterdam,Netherlands,57.302424,4.343602
1,126.555447,Antwerp,Belgium,28.730159,4.404969
2,95.573357,Athens,Greece,36.464309,2.621011
3,310.408633,Austin,United States,69.883248,4.441817
4,77.920512,Bangkok,Thailand,21.727904,3.586196
5,180.903836,Barcelona,Spain,36.409524,4.968586
6,128.786118,Berlin,Germany,43.201905,2.981029
7,143.796868,Bergamo,Italy,21.111111,6.811431
8,254.140615,Bologna,Italy,26.716049,9.512657
9,215.610694,Boston,United States,83.723123,2.575283


In [32]:
#We have a data for 70 cities for define the airbnb factor
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 69
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Entire home/apt  70 non-null     float64
 1   city             70 non-null     object 
 2   country          70 non-null     object 
 3   price_per_day    70 non-null     float64
 4   factor           70 non-null     float64
dtypes: float64(3), object(2)
memory usage: 3.3+ KB


In [26]:
# Put the column factor with values: Daily price from airbnb divided through price per day from numbeo 
merged_df['factor'] = merged_df['Entire home/apt'] / merged_df['price_per_day']

In [29]:
#Check the data
merged_df.head(10)

,Entire home/apt,city,country,price_per_day,factor
0,248.898913,Amsterdam,Netherlands,57.302424,4.343602
1,126.555447,Antwerp,Belgium,28.730159,4.404969
2,95.573357,Athens,Greece,36.464309,2.621011
3,310.408633,Austin,United States,69.883248,4.441817
4,77.920512,Bangkok,Thailand,21.727904,3.586196
5,180.903836,Barcelona,Spain,36.409524,4.968586
6,128.786118,Berlin,Germany,43.201905,2.981029
7,143.796868,Bergamo,Italy,21.111111,6.811431
8,254.140615,Bologna,Italy,26.716049,9.512657
9,215.610694,Boston,United States,83.723123,2.575283


In [26]:
#Now we can use descriptive analysis function to find a median for our factor
merged_df.describe()

,Entire home/apt,price_per_day,factor
count,70.000000,70.000000,70.000000
mean,178.242192,47.482783,4.410715
std,59.696938,24.045550,2.184770
min,58.021089,10.738235,1.999417
25%,132.007833,28.279147,3.055562
50%,174.910400,43.748030,3.932473
75%,217.527752,63.462004,4.815151
max,315.965080,114.092501,16.154458


In [31]:
#Sorting the data ascending and last check
merged_df.sort_values('factor')

,Entire home/apt,city,country,price_per_day,factor
68,228.118510,New York,United States,112.778173,2.022719
52,160.663079,Seattle,United States,74.559228,2.154838
24,149.992455,Hong Kong,Hong Kong (China),68.248258,2.197748
54,242.838383,Singapore,Singapore,108.297395,2.242329
44,126.555052,Portland,United States,53.578747,2.362038
...,...,...,...,...,...
22,232.357377,Florence,Italy,29.102564,7.984086
33,223.570569,Malaga,Spain,27.479417,8.135928
13,182.459755,Cape Town,South Africa,20.148399,9.055794
8,254.140615,Bologna,Italy,26.716049,9.512657


In [34]:
# We drop the outlier Rio de Janeiro and use descriptive analysis function to find a median for our factor
merged_df.drop(47).describe()

,Entire home/apt,price_per_day,factor
count,69.000000,69.000000,69.000000
mean,177.824109,47.985151,4.240515
std,60.030964,23.848800,1.669087
min,58.021089,10.738235,1.999417
25%,131.951993,28.906061,3.047376
50%,174.666805,44.201111,3.890674
75%,217.832603,63.500565,4.784730
max,315.965080,114.092501,9.664016


In [ ]:
# Without Outlier Rio de Janeiro
# Median is 3.89
# Mean/avg is 4.24
# For calc_price we decide to take factor 3.89 